<a href="https://colab.research.google.com/github/Shiva-15711/AI/blob/main/Load_value_Multiple_regression_Assignment_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import required libraries

In [2]:
import numpy as np
import math as mt
import random
import pandas as pd

## Import Data

In [3]:
data = pd.read_csv('/content/drive/MyDrive/AI_data/Fifth_assignment.csv')
data.head()

,X1,X2,X3,Y
0,5551.82208,4983.17184,4888.39680,5072.95872
1,4983.17184,4888.39680,5072.95872,5196.25980
2,4888.39680,5072.95872,5196.25980,5641.29720
3,5072.95872,5196.25980,5641.29720,7153.17732
4,5196.25980,5641.29720,7153.17732,7849.80504


## *Outliers Detection*

In [4]:
outliers = []
def detect_outliers_zscore(data):
    thres = 3
    mean = np.mean(data)
    std = np.std(data)
    # print(mean, std)
    for i in data:
        z_score = (i-mean)/std
        if (np.abs(z_score) > thres):
            outliers.append(i)
    return outliers# Driver code

In [5]:
sample_outliers_X1 = detect_outliers_zscore(data['X1'])
sample_outliers_X2 = detect_outliers_zscore(data['X2'])
sample_outliers_X3 = detect_outliers_zscore(data['X3'])
sample_outliers_Y = detect_outliers_zscore(data['Y'])
print("Outliers in X1 column from Z-scores method: ", sample_outliers_X1)
print("Outliers in X2 column from Z-scores method: ", sample_outliers_X2)
print("Outliers in X3 column from Z-scores method: ", sample_outliers_X3)
print("Outliers in Y column from Z-scores method: ", sample_outliers_Y)

Outliers in X1 column from Z-scores method:  []
Outliers in X2 column from Z-scores method:  []
Outliers in X3 column from Z-scores method:  []
Outliers in Y column from Z-scores method:  []


## Missing Values Treatment

In [6]:
print(data.isnull().sum())

X1    0
X2    1
X3    2
Y     3
dtype: int64


In [7]:
m1=data['X1'].mean()
m2=data['X2'].mean()
m3=data['X3'].mean()
m4=data['Y'].mean()
data['X1'].fillna(value=m1, inplace=True)
data['X2'].fillna(value=m2, inplace=True)
data['X3'].fillna(value=m3, inplace=True)
data['Y'].fillna(value=m4, inplace=True)
data

,X1,X2,X3,Y
0,5551.82208,4983.171840,4888.396800,5072.958720
1,4983.17184,4888.396800,5072.958720,5196.259800
2,4888.39680,5072.958720,5196.259800,5641.297200
3,5072.95872,5196.259800,5641.297200,7153.177320
4,5196.25980,5641.297200,7153.177320,7849.805040
...,...,...,...,...
2179,4626.51840,4567.907520,4723.631640,5150.275200
2180,4567.90752,4723.631640,5150.275200,6938.374680
2181,4723.63164,5150.275200,6938.374680,6029.345387
2182,5150.27520,6938.374680,6028.822496,6029.345387


# Min and max values

In [8]:
x1_min=data['X1'].min()
x1_max=data['X1'].max()
x2_min=data['X2'].min()
x2_max=data['X2'].max()
x3_min=data['X3'].min()
x3_max=data['X3'].max()
y_min=data['Y'].min()
y_max=data['Y'].max()
print(x1_min, x1_max, x2_min, x2_max, x3_min, x3_max, y_min, y_max )

3377.9196 8841.66948 3377.9196 8841.66948 3377.9196 8841.66948 3377.9196 8841.66948


# Normalization

In [9]:

from sklearn.preprocessing import MinMaxScaler
X1 = np.array(data['X1']).reshape(-1,1)
scaler = MinMaxScaler()
scaler.fit(X1)
X1_scaled = scaler.transform(X1)
data['X1'] = X1_scaled.reshape(1,-1)[0]

X2 = np.array(data['X2']).reshape(-1,1)
scaler = MinMaxScaler()
scaler.fit(X2)
X2_scaled = scaler.transform(X2)
data['X2'] = X2_scaled.reshape(1,-1)[0]

X3 = np.array(data['X3']).reshape(-1,1)
scaler = MinMaxScaler()
scaler.fit(X3)
X3_scaled = scaler.transform(X3)
data['X3'] = X3_scaled.reshape(1,-1)[0]

In [10]:
Y1 = np.array(data['Y']).reshape(-1,1)
scaler = MinMaxScaler()
scaler.fit(Y1)
Y_scaled = scaler.transform(Y1)
data['Y'] = Y_scaled.reshape(1,-1)[0]

## Data Split

In [11]:
from sklearn.model_selection import train_test_split
dataframe = data.values
X, y = dataframe[:, :-1], dataframe[:, -1]
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.10, random_state = 10)
x_train

array([[0.44840375, 0.46706228, 0.50332373],
       [0.6971841 , 0.82468403, 0.68785484],
       [0.72206214, 0.62565975, 0.37613763],
       ...,
       [0.61011098, 0.44563636, 0.4770192 ],
       [0.22641294, 0.17160709, 0.20062195],
       [0.4363071 , 0.41339762, 0.37793501]])

# Initialization

In [12]:
m1=random.uniform(-10,10)
m2=random.uniform(-10,10)
m3=random.uniform(-10,10)
c=random.uniform(-10,10)
eta=0.1
max_iter=10000

## Gradient Calculation

In [13]:
def gradient(m1,m2,m3,c,x1,x2,x3,y):
  m1_deriv=-1*(y-m1*x1-m2*x2-m3*x3-c)*x1
  m2_deriv=-1*(y-m1*x1-m2*x2-m3*x3-c)*x2
  m3_deriv=-1*(y-m1*x1-m2*x2-m3*x3-c)*x3
  c_deriv= -1*(y-m1*x1-m2*x2-m3*x3-c)
  return m1_deriv,m2_deriv,m3_deriv,c_deriv

## Training Using Stochastic Gradient Descent Optimizer

In [15]:
for iter in range(max_iter):
  for sample in range(x_train.shape[0]):
    del_m1,del_m2,del_m3,del_c=gradient(m1,m2,m3,c, x_train[sample,0],x_train[sample,1],x_train[sample,2],y_train[sample])
    m1=m1-eta*del_m1
    m2=m2-eta*del_m2
    m3=m3-eta*del_m3
    c=c-eta*del_c
print(f'Model parameters m1=', m1,'m2=',m2,'m3=',m3,'and c=',c)

Model parameters m1= -0.2063114917803206 m2= 0.03313104115540776 m3= 0.8517675102387888 and c= 0.14590702006987477


# Model Prediction on Training Data





In [16]:
y_pred_train = []
for sample in range(x_train.shape[0]):
  x1=x_train[sample,0]
  x2=x_train[sample,1]
  x3=x_train[sample,2]
  y=y_train[sample]
  y_p_tr = (m1 * x1) + (m2 * x2) + (m3*x3) + c 
  y_pred_train.append(y_p_tr)
y_pred_train = np.array(y_pred_train)

# Model Prediction on Testing Data





In [17]:
y_pred_test = []
for sample in range(x_test.shape[0]):
  x1=x_test[sample,0]
  x2=x_test[sample,1]
  x3=x_test[sample,2]
  y=y_test[sample]
  y_p_te = (m1 * x1) + (m2 * x2) + (m3*x3) + c 
  y_pred_test.append(y_p_te)
y_pred_test = np.array(y_pred_test)

# Error Metrics Calculation

In [18]:

import math
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error


#Training Accuracies
rmse = math.sqrt(mean_squared_error(y_train, y_pred_train)) 
print('Root mean square error', rmse) 
mse = (mean_squared_error(y_train, y_pred_train)) 
print('Mean square error', mse) 
mae=mean_absolute_error(y_train, y_pred_train)
print('Mean absolute error', mae)

Root mean square error 0.11641604321824466
Mean square error 0.013552695118592209
Mean absolute error 0.08034661092624044


In [19]:
#Testing Accuracies
mse = math.sqrt(mean_squared_error(y_test, y_pred_test)) 
print('Root mean square error', mse) 
mse = (mean_squared_error(y_test, y_pred_test)) 
print('Mean square error', mse) 
mae=mean_absolute_error(y_test, y_pred_test)
print('Mean absolute error', mae)

Root mean square error 0.12655266836682572
Mean square error 0.016015577870763773
Mean absolute error 0.08626953972716382


# Deployment

In [20]:
X1=input('Enter the [t-3] load value (3 hours before current time ')
X2=input('Enter the [t-2] load value (2 hours before current time ')
X3=input('Enter the [t-1] load value (1 hour before current time ')
X1=float(X1)
X2=float(X2)
X3=float(X3)  
print('X1=',X1)
print('X2=',X2)
print('X3=',X3)
x1_nor=(X1 - x1_min)/(x1_max - x1_min)
x2_nor=(X2 - x2_min)/(x2_max - x2_min)
x3_nor=(X3 - x3_min)/(x3_max - x3_min)
y_nor = m1*x1_nor + m2*x2_nor + m3*x3_nor + c
Y = y_nor * (y_max-y_min) + y_min
print('Load value at [t] present time = ',Y)

Enter the [t-3] load value (3 hours before current time 2
Enter the [t-2] load value (2 hours before current time 3
Enter the [t-1] load value (1 hour before current time 1
X1= 2.0
X2= 3.0
X3= 1.0
Load value at [t] present time =  1883.4450720720001


# Thank You